# LangChain ChatOpenAI + PydanticOutputParser 예제
이 노트북은 LangChain의 `ChatOpenAI`와 `PydanticOutputParser`를 사용하여 구조화된 출력을 생성하는 방법을 보여줍니다.

In [3]:
# 필수 라이브러리 설치
!poetry add pydantic

Using version ^2.11.9 for pydantic

Updating dependencies
Resolving dependencies...

No dependencies to install or update

Writing lock file


In [4]:
!poetry show pydantic

 name         : pydantic                                
 version      : 2.11.9                                  
 description  : Data validation using Python type hints 

dependencies
 - annotated-types >=0.6.0
 - pydantic-core 2.33.2
 - typing-extensions >=4.12.2
 - typing-inspection >=0.4.0

required by
 - fastapi requires >=1.7.4,<1.8 || >1.8,<1.8.1 || >1.8.1,<2.0.0 || >2.0.0,<2.0.1 || >2.0.1,<2.1.0 || >2.1.0,<3.0.0
 - gradio requires >=2.0,<2.12
 - langchain requires >=2.7.4,<3.0.0
 - langchain-core requires >=2.7.4
 - langsmith requires >=1,<3
 - openai requires >=1.9.0,<3


In [10]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

In [18]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")

In [19]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

In [20]:
# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [26]:
# 환경변수에서 OpenAI API 키 로드 (실제 사용시 주석 해제)
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo") #<< 안쓰임
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [46]:
# 체인 구성 및 실행
query = "1990년대 부터 2000년대의 동아시아에서 상영된 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

pprint(type(output))
pprint(output)

<class '__main__.MovieRecommendation'>
MovieRecommendation(movie_title='린(링)(リング)', reason='1990~2000년대 동아시아 공포의 상징적 작품. 비디오 테이프를 본 사람이 일주일 뒤 죽는다는 설정이 신선했고, 미사키 요코(貞子) 캐릭터가 한국·일본 공포 아이콘이 되었다. 일본판 1998년작 원작의 독특한 분위기와 긴장감 넘치는 연출은 지금 봐도 강렬하다.', genre=['공포', '미스터리', '초자연'], estimated_rating=8.7)


In [47]:
# 결과 출력
pprint(f"추천 영화: {output.movie_title}")
pprint(f"추천 이유: {output.reason}")
pprint(f"장르: {', '.join(output.genre)}")
pprint(f"예상 평점: {output.estimated_rating}/10")

'추천 영화: 린(링)(リング)'
('추천 이유: 1990~2000년대 동아시아 공포의 상징적 작품. 비디오 테이프를 본 사람이 일주일 뒤 죽는다는 설정이 신선했고, 미사키 '
 '요코(貞子) 캐릭터가 한국·일본 공포 아이콘이 되었다. 일본판 1998년작 원작의 독특한 분위기와 긴장감 넘치는 연출은 지금 봐도 '
 '강렬하다.')
'장르: 공포, 미스터리, 초자연'
'예상 평점: 8.7/10'


## 예상 출력 결과
```
추천 영화: The Silence of the Lambs
추천 이유: 1991년에 개봉한 이 영화는 심리적 공포의 걸작으로 평가받으며, 1990년대를 대표하는 영화 중 하나입니다.
장르: thriller, horror, crime
예상 평점: 8.6/10
```